# Caching

- Armazena resultados de consultas e cálculos para reutilização futura
- **Melhora Performance**: Reduz o tempo de processamento reutilizando resultados já calculados
- **Economia de Custos**: Minimiza chamadas repetidas a APIs
- **Eficiência**: Diminui a carga do sistema, permitindo um uso mais eficiente dos recursos.

Vamos ver três tipos de caching aqui:
- **Memória**
- **Disco/Banco de Dados**
- **Personalizado**

# Setup

In [1]:
from langchain_openai import OpenAI
from langchain_community.cache import InMemoryCache, SQLiteCache
from langchain_core.globals import set_llm_cache
from dotenv import load_dotenv

load_dotenv()

True

# Memória

In [2]:
openai = OpenAI(model_name='gpt-3.5-turbo-instruct')
set_llm_cache(InMemoryCache())

In [3]:
prompt = 'Me diga em poucas palavras quem foi Carl Segan.'
response1 = openai.invoke(prompt)
print('Primeira resposta (API chamada):', response1)

Primeira resposta (API chamada): 

Carl Sagan foi um renomado astrônomo, astrofísico, cosmólogo, escritor e divulgador científico norte-americano. Ele é conhecido por seu trabalho na pesquisa e exploração do sistema solar, além de ter sido pioneiro na busca por vida extraterrestre. Sagan também foi um defensor da ciência e do ceticismo, tendo popularizado temas complexos da astronomia e cosmologia para o grande público através de seus livros, programas de televisão e palestras. Ele é considerado um dos cientistas mais influentes e importantes do século XX. 


In [4]:
response2 = openai.invoke(prompt)
print('Segunda resposta (usando cache):', response2)

Segunda resposta (usando cache): 

Carl Sagan foi um renomado astrônomo, astrofísico, cosmólogo, escritor e divulgador científico norte-americano. Ele é conhecido por seu trabalho na pesquisa e exploração do sistema solar, além de ter sido pioneiro na busca por vida extraterrestre. Sagan também foi um defensor da ciência e do ceticismo, tendo popularizado temas complexos da astronomia e cosmologia para o grande público através de seus livros, programas de televisão e palestras. Ele é considerado um dos cientistas mais influentes e importantes do século XX. 


# Disco / Banco de Dados

In [5]:
set_llm_cache(SQLiteCache(database_path='openai_cache.db'))

In [6]:
prompt = 'Me diga em poucas quem foi Neil Armstrong'

In [7]:
response1 = openai.invoke(prompt)
print('Primeira resposta (API chamada):', response1)

Primeira resposta (API chamada): 

Neil Armstrong foi um astronauta, engenheiro aeronáutico e piloto norte-americano, conhecido por ser o primeiro ser humano a pisar na Lua, em 20 de julho de 1969, durante a missão Apollo 11. Ele ficou famoso por sua famosa frase "Este é um pequeno passo para o homem, mas um salto gigante para a humanidade". Armstrong também participou de outras missões espaciais e foi um herói nacional nos Estados Unidos. Ele faleceu em 2012, aos 82 anos, deixando um legado de coragem, determinação e avanços na exploração espacial. 


In [8]:
response2 = openai.invoke(prompt)
print('Segunda resposta (usando cache):', response2)

Segunda resposta (usando cache): 

Neil Armstrong foi um astronauta, engenheiro aeronáutico e piloto norte-americano, conhecido por ser o primeiro ser humano a pisar na Lua, em 20 de julho de 1969, durante a missão Apollo 11. Ele ficou famoso por sua famosa frase "Este é um pequeno passo para o homem, mas um salto gigante para a humanidade". Armstrong também participou de outras missões espaciais e foi um herói nacional nos Estados Unidos. Ele faleceu em 2012, aos 82 anos, deixando um legado de coragem, determinação e avanços na exploração espacial. 


Perceba que, a resposta do prompt foi guardada no banco de dados e, chamando o mesmo prompt, ao invés de gerar uma nova chamada, o LangChain buscou a resposta no banco de dados.

Do jeito que fizemos, a resposta só vai ser reciclada se o prompt for **exatamente** o mesmo. Vamos resoler isso futuramente mas, por enquanto, vamos nos contentar com isso.

# Personalizado


In [10]:
import os
import json
import hashlib

class SimpleDiskCache:

    def __init__(self, cache_dir='cache_dir'):
        self.cache_dir = cache_dir
        os.makedirs(self.cache_dir, exist_ok=True)

    def _get_cache_path(self, key):
        hashed_key = hashlib.md5(key.encode()).hexdigest() # hash cria nome único do arquivo
        return os.path.join(self.cache_dir, f'{hashed_key}.json')
    
    def lookup(self, key, llm_string):
        cache_path = self._get_cache_path(key)
        if os.path.exists(cache_path):
            with open(cache_path, 'r') as f:
                return json.load(f)
        return None
    
    def update(self, key, value, llm_string):
        cache_path = self._get_cache_path(key)
        with open(cache_path, 'w') as f:
            json.dump(value, f)

O **LangChain** vai procurar pelos métodos **lookup** e **update** então esses métodos são obrigatórios na classe

In [11]:
cache = SimpleDiskCache()
set_llm_cache(cache)
prompt = 'Me diga em poucas palavras quem foi Degrasse Tyson'

In [16]:
def invoke_with_cache(llm, prompt, cache):
    cached_response = cache.lookup(prompt, ' ')
    if cached_response:
        print('Usando chace:')
        return cached_response
    
    response = llm.invoke(prompt)
    cache.update(prompt, response, ' ')
    return response

In [17]:
response1 = invoke_with_cache(openai, prompt, cache)
response_text1 = response1.replace('\n', ' ')

print('Primeira resposta (API chamada):', response_text1)

Usando chace:
Primeira resposta (API chamada):   Neil Degrasse Tyson é um astrofísico, cosmólogo e divulgador científico americano, conhecido por seu trabalho em popularizar a astronomia e por ser o diretor do Planetário Hayden em Nova York. Ele também é conhecido por apresentar e narrar programas de televisão sobre ciência, e por suas contribuições na pesquisa e comunicação sobre o universo e a exploração espacial. 


In [18]:
response2 = invoke_with_cache(openai, prompt, cache)
response_text2 = response2.replace('\n', ' ')

print('Segunda resposta (usando cache):', response_text2)

Usando chace:
Segunda resposta (usando cache):   Neil Degrasse Tyson é um astrofísico, cosmólogo e divulgador científico americano, conhecido por seu trabalho em popularizar a astronomia e por ser o diretor do Planetário Hayden em Nova York. Ele também é conhecido por apresentar e narrar programas de televisão sobre ciência, e por suas contribuições na pesquisa e comunicação sobre o universo e a exploração espacial. 
